Q1. As a part of the assessment you are supposed to analyse the logs from https://github.com/logpai/loghub/tree/master/HealthApp and build analytics solutions i.e. when does the person work-out, how many steps he took on daily basis, what are his active times etc. The more analytics you generate better it will be for evaluation. Analytics should have better visualisation in form of graphs. Use Python or Java as the language of choice

In [30]:
import pandas as pd
import numpy as np
import re

In [31]:
df = pd.read_csv('HealthApp_2k.log_structured.csv')

In [32]:
df

,LineId,Time,Component,Pid,Content,EventId,EventTemplate
0,1,20171223-22:15:29:606,Step_LSC,30002312,onStandStepChanged 3579,E42,onStandStepChanged <*>
1,2,20171223-22:15:29:615,Step_LSC,30002312,onExtend:1514038530000 14 0 4,E39,onExtend:<*> <*> <*> <*>
2,3,20171223-22:15:29:633,Step_StandReportReceiver,30002312,onReceive action: android.intent.action.SCREEN_ON,E41,onReceive action: android.intent.action.SCREEN_ON
3,4,20171223-22:15:29:635,Step_LSC,30002312,processHandleBroadcastAction action:android.in...,E43,processHandleBroadcastAction action:android.in...
4,5,20171223-22:15:29:635,Step_StandStepCounter,30002312,flush sensor data,E12,flush sensor data
...,...,...,...,...,...,...,...
1995,1996,20171224-0:58:53:985,Step_LSC,30002312,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1996,1997,20171224-0:59:7:581,Step_LSC,30002312,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1997,1998,20171224-1:0:0:794,Step_LSC,30002312,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1998,1999,20171224-1:1:0:935,Step_LSC,30002312,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...


In [33]:
mf=pd.read_csv('HealthApp_2k.log_templates.csv')

In [34]:
mf

,EventId,EventTemplate
0,E1,Alarm uploadStaticsToDB totalSteps=<*>:<*>:<*>...
1,E2,"bulkSaveDetailHiHealthData() size = <*>,totalT..."
2,E3,calculateAltitudeWithCache totalAltitude=<*>
3,E4,calculateCaloriesWithCache totalCalories=<*>
4,E5,"checkCurrentDay a new day comes , reset basicS..."
...,...,...
70,E71,tryToReloadTodayBasicSteps<*>|<*>|<*>|<*>
71,E72,"upLoadOneMinuteDataToEngine time=<*>,<*>,<*>,<..."
72,E73,uploadStaticsToDB failed message=true
73,E74,uploadStaticsToDB() onResult type = <*> obj=true


In [35]:
unique_values = df['Pid'].unique()
unique_values

array([30002312], dtype=int64)

In [36]:
df['Component'].unique()

array(['Step_LSC', 'Step_StandReportReceiver', 'Step_StandStepCounter',
       'Step_SPUtils', 'Step_ExtSDM', 'Step_ScreenUtil',
       'Step_FlushableStepDataCache', 'HiH_HiAppUtil',
       'HiH_HiHealthBinder', 'HiH_HiHealthDataInsertStore',
       'HiH_DataStatManager', 'HiH_ListenerManager', 'HiH_HiSyncControl',
       'HiH_HiSyncUtil', 'HiH_', 'HiH_HiBroadcastUtil',
       'Step_StandStepDataManager', 'Step_DataCache', 'Step_HGNH',
       'Step_NotificationUtil'], dtype=object)

In [37]:
# Content
df['Content'].unique()

array(['onStandStepChanged 3579', 'onExtend:1514038530000 14 0 4',
       'onReceive action: android.intent.action.SCREEN_ON', ...,
       ' getTodayTotalDetailSteps = 1514046240000##0##549659##8661##16256##34961757',
       'setTodayTotalDetailSteps=1514046420000##0##549659##8661##16256##35149773',
       'checkCurrentDay a new day comes , reset basicSyncCondition, currentDay is 20171224 oldDay is 20171223'],
      dtype=object)

In [38]:
# EventTemplate
df['EventTemplate'].unique()

array(['onStandStepChanged <*>', 'onExtend:<*> <*> <*> <*>',
       'onReceive action: android.intent.action.SCREEN_ON',
       'processHandleBroadcastAction action:android.intent.action.SCREEN_ON',
       'flush sensor data',
       'getTodayTotalDetailSteps = <*>##<*>##<*>##<*>##<*>##<*>',
       'setTodayTotalDetailSteps=<*>',
       'calculateCaloriesWithCache totalCalories=<*>',
       'calculateAltitudeWithCache totalAltitude=<*>',
       'REPORT : <*> <*> <*> <*>',
       'onReceive action: android.intent.action.SCREEN_OFF',
       'isScreenOn true', 'screen status unknown,think screen on',
       'processHandleBroadcastAction action:android.intent.action.TIME_TICK',
       'flushTempCacheToDB by stand',
       'Alarm uploadStaticsToDB totalSteps=<*>:<*>:<*>:<*>',
       'writeDataToDB size <*>',
       'upLoadOneMinuteDataToEngine time=<*>,<*>,<*>,<*>,<*>',
       'getBinderPackageName packageName = com.huawei.health',
       'getAppContext() isAppValid health or wear, packageN

In [39]:
df = df.drop(['LineId','Pid'], axis=1)

In [40]:
df

,Time,Component,Content,EventId,EventTemplate
0,20171223-22:15:29:606,Step_LSC,onStandStepChanged 3579,E42,onStandStepChanged <*>
1,20171223-22:15:29:615,Step_LSC,onExtend:1514038530000 14 0 4,E39,onExtend:<*> <*> <*> <*>
2,20171223-22:15:29:633,Step_StandReportReceiver,onReceive action: android.intent.action.SCREEN_ON,E41,onReceive action: android.intent.action.SCREEN_ON
3,20171223-22:15:29:635,Step_LSC,processHandleBroadcastAction action:android.in...,E43,processHandleBroadcastAction action:android.in...
4,20171223-22:15:29:635,Step_StandStepCounter,flush sensor data,E12,flush sensor data
...,...,...,...,...,...
1995,20171224-0:58:53:985,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1996,20171224-0:59:7:581,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1997,20171224-1:0:0:794,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1998,20171224-1:1:0:935,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...


In [41]:
merged_df = df.merge(mf, on="EventId", how="inner")

In [42]:
merged_df

,Time,Component,Content,EventId,EventTemplate_x,EventTemplate_y
0,20171223-22:15:29:606,Step_LSC,onStandStepChanged 3579,E42,onStandStepChanged <*>,onStandStepChanged <*>
1,20171223-22:15:29:636,Step_LSC,onStandStepChanged 3579,E42,onStandStepChanged <*>,onStandStepChanged <*>
2,20171223-22:15:29:738,Step_LSC,onStandStepChanged 3579,E42,onStandStepChanged <*>,onStandStepChanged <*>
3,20171223-22:15:29:792,Step_LSC,onStandStepChanged 3580,E42,onStandStepChanged <*>,onStandStepChanged <*>
4,20171223-22:15:30:331,Step_LSC,onStandStepChanged 3581,E42,onStandStepChanged <*>,onStandStepChanged <*>
...,...,...,...,...,...,...
1995,20171224-0:0:0:237,Step_HGNH,next day:steps0mLastReport7214,E38,next day:<*>,next day:<*>
1996,20171224-0:0:0:237,Step_SPUtils,setGoalNotifiShownRecord 1514044800237 false,E57,setGoalNotifiShownRecord <*> false,setGoalNotifiShownRecord <*> false
1997,20171224-0:0:0:240,Step_HGNH,closeNotification...,E10,closeNotification...,closeNotification...
1998,20171224-0:0:0:240,Step_NotificationUtil,deleteHealthNotification(),E11,deleteHealthNotification(),deleteHealthNotification()


In [43]:
df.head(50)

,Time,Component,Content,EventId,EventTemplate
0,20171223-22:15:29:606,Step_LSC,onStandStepChanged 3579,E42,onStandStepChanged <*>
1,20171223-22:15:29:615,Step_LSC,onExtend:1514038530000 14 0 4,E39,onExtend:<*> <*> <*> <*>
2,20171223-22:15:29:633,Step_StandReportReceiver,onReceive action: android.intent.action.SCREEN_ON,E41,onReceive action: android.intent.action.SCREEN_ON
3,20171223-22:15:29:635,Step_LSC,processHandleBroadcastAction action:android.in...,E43,processHandleBroadcastAction action:android.in...
4,20171223-22:15:29:635,Step_StandStepCounter,flush sensor data,E12,flush sensor data
5,20171223-22:15:29:635,Step_SPUtils,getTodayTotalDetailSteps = 1514038440000##699...,E22,getTodayTotalDetailSteps = <*>##<*>##<*>##<*>#...
6,20171223-22:15:29:636,Step_SPUtils,setTodayTotalDetailSteps=1514038440000##7007##...,E58,setTodayTotalDetailSteps=<*>
7,20171223-22:15:29:636,Step_LSC,onStandStepChanged 3579,E42,onStandStepChanged <*>
8,20171223-22:15:29:645,Step_ExtSDM,calculateCaloriesWithCache totalCalories=126775,E4,calculateCaloriesWithCache totalCalories=<*>
9,20171223-22:15:29:648,Step_ExtSDM,calculateAltitudeWithCache totalAltitude=240,E3,calculateAltitudeWithCache totalAltitude=<*>


In [44]:
df.tail(60)

,Time,Component,Content,EventId,EventTemplate
1940,20171224-0:17:0:126,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1941,20171224-0:18:0:103,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1942,20171224-0:19:0:137,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1943,20171224-0:20:0:128,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1944,20171224-0:21:0:131,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1945,20171224-0:22:0:135,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1946,20171224-0:23:0:141,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1947,20171224-0:24:0:114,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1948,20171224-0:25:0:132,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction action:android.in...
1949,20171224-0:25:16:137,Step_LSC,onStandStepChanged 3786,E42,onStandStepChanged <*>


In [45]:

def remove_symbols(text):
    return re.sub(r'[^\w\s]', '', text)


In [46]:
df['EventTemplate'] = df['EventTemplate'].apply(remove_symbols)

In [47]:
df

,Time,Component,Content,EventId,EventTemplate
0,20171223-22:15:29:606,Step_LSC,onStandStepChanged 3579,E42,onStandStepChanged
1,20171223-22:15:29:615,Step_LSC,onExtend:1514038530000 14 0 4,E39,onExtend
2,20171223-22:15:29:633,Step_StandReportReceiver,onReceive action: android.intent.action.SCREEN_ON,E41,onReceive action androidintentactionSCREEN_ON
3,20171223-22:15:29:635,Step_LSC,processHandleBroadcastAction action:android.in...,E43,processHandleBroadcastAction actionandroidinte...
4,20171223-22:15:29:635,Step_StandStepCounter,flush sensor data,E12,flush sensor data
...,...,...,...,...,...
1995,20171224-0:58:53:985,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction actionandroidinte...
1996,20171224-0:59:7:581,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction actionandroidinte...
1997,20171224-1:0:0:794,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction actionandroidinte...
1998,20171224-1:1:0:935,Step_LSC,processHandleBroadcastAction action:android.in...,E44,processHandleBroadcastAction actionandroidinte...


In [48]:
df['EventTemplate'].unique()

array(['onStandStepChanged ', 'onExtend   ',
       'onReceive action androidintentactionSCREEN_ON',
       'processHandleBroadcastAction actionandroidintentactionSCREEN_ON',
       'flush sensor data', 'getTodayTotalDetailSteps  ',
       'setTodayTotalDetailSteps',
       'calculateCaloriesWithCache totalCalories',
       'calculateAltitudeWithCache totalAltitude', 'REPORT     ',
       'onReceive action androidintentactionSCREEN_OFF',
       'isScreenOn true', 'screen status unknownthink screen on',
       'processHandleBroadcastAction actionandroidintentactionTIME_TICK',
       'flushTempCacheToDB by stand',
       'Alarm uploadStaticsToDB totalSteps', 'writeDataToDB size ',
       'upLoadOneMinuteDataToEngine time',
       'getBinderPackageName packageName  comhuaweihealth',
       'getAppContext isAppValid health or wear packageName  ',
       'insertHiHealthData checkAppType   appID  ',
       'insertHiHealthData size  app  One Data Type  packageName  writeStatType  ',
       's

In [49]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Length: 2000, dtype: bool

In [50]:
keywords_to_keep = [
    "onStandStepChanged",
    "onExtend",
    "getTodayTotalDetailSteps",
    "setTodayTotalDetailSteps",
    "calculateCaloriesWithCache totalCalories",
    "calculateAltitudeWithCache totalAltitude",
    "REPORT"
]

# Function to check if any keyword is present in a row
def contains_keywords(row):
    return any(keyword in row for keyword in keywords_to_keep)

In [51]:
# Apply the filter and create a new DataFrame
filtered_df = df[df['Content'].apply(contains_keywords)]

In [56]:
filtered_df.tail(50)

,Time,Component,Content,EventId,EventTemplate
1902,20171224-0:10:10:446,Step_ExtSDM,calculateCaloriesWithCache totalCalories=0,E4,calculateCaloriesWithCache totalCalories
1903,20171224-0:10:10:446,Step_ExtSDM,calculateAltitudeWithCache totalAltitude=0,E3,calculateAltitudeWithCache totalAltitude
1904,20171224-0:10:42:137,Step_LSC,onStandStepChanged 3786,E42,onStandStepChanged
1905,20171224-0:10:42:149,Step_LSC,onExtend:1514045442000 0 0 5,E39,onExtend
1906,20171224-0:10:42:438,Step_SPUtils,getTodayTotalDetailSteps = 1514045340000##0##...,E22,getTodayTotalDetailSteps
1907,20171224-0:10:42:438,Step_SPUtils,setTodayTotalDetailSteps=1514045340000##0##549...,E58,setTodayTotalDetailSteps
1908,20171224-0:10:42:445,Step_ExtSDM,calculateCaloriesWithCache totalCalories=0,E4,calculateCaloriesWithCache totalCalories
1909,20171224-0:10:42:445,Step_ExtSDM,calculateAltitudeWithCache totalAltitude=0,E3,calculateAltitudeWithCache totalAltitude
1910,20171224-0:10:43:138,Step_LSC,onStandStepChanged 3786,E42,onStandStepChanged
1911,20171224-0:10:43:153,Step_LSC,onExtend:1514045443000 0 0 0,E39,onExtend


In [57]:
filtered_df['Content'].duplicated()

0       False
1       False
5       False
6       False
7        True
        ...  
1965    False
1966    False
1967    False
1968     True
1969     True
Name: Content, Length: 1634, dtype: bool

In [63]:
newdf = pd.DataFrame(filtered_df['Content'])

In [64]:
newdf

,Content
0,onStandStepChanged 3579
1,onExtend:1514038530000 14 0 4
5,getTodayTotalDetailSteps = 1514038440000##699...
6,setTodayTotalDetailSteps=1514038440000##7007##...
7,onStandStepChanged 3579
...,...
1965,onExtend:1514046505000 0 0 5
1966,getTodayTotalDetailSteps = 1514046240000##0##...
1967,setTodayTotalDetailSteps=1514046420000##0##549...
1968,calculateCaloriesWithCache totalCalories=0


In [65]:
def extract_columns_from_content(df):
    # Define patterns to search for in 'Content'
    patterns = {
        "onStandStepChanged": r'onStandStepChanged (\d+)',
        "onExtend": r'onExtend:(\d+)',
        "getTodayTotalDetailSteps": r'getTodayTotalDetailSteps = (.+)',
        "setTodayTotalDetailSteps": r'setTodayTotalDetailSteps=(.+)',
        "calculateCaloriesWithCache totalCalories": r'calculateCaloriesWithCache totalCalories=(\d+)',
        "calculateAltitudeWithCache totalAltitude": r'calculateAltitudeWithCache totalAltitude=(\d+)',
        "REPORT.Content": r'REPORT : (.+)'
    }

    extracted_data = {}

    for col_name, pattern in patterns.items():
        extracted_data[col_name] = newdf['Content'].str.extract(pattern, expand=False)


    new_df = pd.DataFrame(extracted_data)

    return new_df

In [66]:
new_df = extract_columns_from_content(df)

In [67]:
new_df

,onStandStepChanged,onExtend,getTodayTotalDetailSteps,setTodayTotalDetailSteps,calculateCaloriesWithCache totalCalories,calculateAltitudeWithCache totalAltitude,REPORT.Content
0,3579,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1514038530000,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,1514038440000##6993##548365##8661##12266##2716...,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,1514038440000##7007##548365##8661##12361##2717...,NaN,NaN,NaN
7,3579,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1965,NaN,1514046505000,NaN,NaN,NaN,NaN,NaN
1966,NaN,NaN,1514046240000##0##549659##8661##16256##34961757,NaN,NaN,NaN,NaN
1967,NaN,NaN,NaN,1514046420000##0##549659##8661##16256##35149773,NaN,NaN,NaN
1968,NaN,NaN,NaN,NaN,0,NaN,NaN


In [68]:
def extract_data_above_report(df):
    extracted_data = []
    data_buffer = []

    for row in filtered_df['Content']:
        if "REPORT" in row:
            if data_buffer:
                extracted_data.append(tuple(data_buffer))
                data_buffer = []
        else:
            data_buffer.append(row)

    return extracted_data

data_above_report = extract_data_above_report(filtered_df)

In [79]:
for i, data_tuple in enumerate(data_above_report, start=1):
    print(f"Tuple {i}:")
    print(data_tuple)
    print()

Tuple 1:
('onStandStepChanged 3579', 'onExtend:1514038530000 14 0 4', ' getTodayTotalDetailSteps = 1514038440000##6993##548365##8661##12266##27164404', 'setTodayTotalDetailSteps=1514038440000##7007##548365##8661##12361##27173954', 'onStandStepChanged 3579', 'calculateCaloriesWithCache totalCalories=126775', 'calculateAltitudeWithCache totalAltitude=240')

Tuple 2:
('onExtend:1514038530000 0 0 4', 'onStandStepChanged 3579', 'onStandStepChanged 3580', 'onExtend:1514038530000 1 0 4', ' getTodayTotalDetailSteps = 1514038440000##7007##548365##8661##12361##27173954', 'setTodayTotalDetailSteps=1514038440000##7008##548365##8661##12456##27174269', 'calculateCaloriesWithCache totalCalories=126797', 'calculateAltitudeWithCache totalAltitude=240')

Tuple 3:
('onStandStepChanged 3581', 'onExtend:1514038531000 1 0 4', ' getTodayTotalDetailSteps = 1514038440000##7008##548365##8661##12456##27174269', 'setTodayTotalDetailSteps=1514038440000##7009##548365##8661##12551##27174951', 'calculateCaloriesWithC

In [80]:
data_above_report

[('onStandStepChanged 3579',
  'onExtend:1514038530000 14 0 4',
  ' getTodayTotalDetailSteps = 1514038440000##6993##548365##8661##12266##27164404',
  'setTodayTotalDetailSteps=1514038440000##7007##548365##8661##12361##27173954',
  'onStandStepChanged 3579',
  'calculateCaloriesWithCache totalCalories=126775',
  'calculateAltitudeWithCache totalAltitude=240'),
 ('onExtend:1514038530000 0 0 4',
  'onStandStepChanged 3579',
  'onStandStepChanged 3580',
  'onExtend:1514038530000 1 0 4',
  ' getTodayTotalDetailSteps = 1514038440000##7007##548365##8661##12361##27173954',
  'setTodayTotalDetailSteps=1514038440000##7008##548365##8661##12456##27174269',
  'calculateCaloriesWithCache totalCalories=126797',
  'calculateAltitudeWithCache totalAltitude=240'),
 ('onStandStepChanged 3581',
  'onExtend:1514038531000 1 0 4',
  ' getTodayTotalDetailSteps = 1514038440000##7008##548365##8661##12456##27174269',
  'setTodayTotalDetailSteps=1514038440000##7009##548365##8661##12551##27174951',
  'calculateCal

In [84]:
column_names = [
    "onStandStepChanged",
    "onExtend",
    "getTodayTotalDetailSteps",
    "setTodayTotalDetailSteps",
    "calculateCaloriesWithCache totalCalories",
    "calculateAltitudeWithCache totalAltitude"
]
Tf = pd.DataFrame(data_above_report, columns=column_names)

ValueError: 6 columns passed, passed data had 252 columns

In [82]:
Tf

NameError: name 'Tf' is not defined

In [71]:
def extract_data_above_report(df):
    extracted_data = []
    data_buffer = []
    row_ids = []

    for row_id, row in enumerate(filtered_df['Content'], start=1):
        if "REPORT" in row:
            if data_buffer:
                extracted_data.append(tuple(data_buffer))
                row_ids.append(row_id)
                data_buffer = []
        else:
            data_buffer.append(row)

    return pd.DataFrame({'RowID': row_ids, 'Data': extracted_data})

In [72]:
data_above_report_df = extract_data_above_report(filtered_df)
data_above_report_df

,RowID,Data
0,8,"(onStandStepChanged 3579, onExtend:15140385300..."
1,17,"(onExtend:1514038530000 0 0 4, onStandStepChan..."
2,24,"(onStandStepChanged 3581, onExtend:15140385310..."
3,31,"(onStandStepChanged 3583, onExtend:15140385310..."
4,38,"(onStandStepChanged 3584, onExtend:15140385320..."
...,...,...
131,1201,"(onStandStepChanged 3784, onExtend:15140420870..."
132,1208,"(onStandStepChanged 3786, onExtend:15140420880..."
133,1229,"(onStandStepChanged 3786, onExtend:15140420910..."
134,1243,"(onStandStepChanged 3786, onExtend:15140422610..."
